In [26]:
import os
import pickle
import numpy as np

In [11]:
data_names = os.listdir("results/")
data_names.remove(".DS_Store")
print(data_names)

['house_8L', 'fried', 'energy', 'kin8nm', 'cd_friedman1_global_abrupt', '2dplanes', 'newsPopularity', 'puma8NH', 'elevators', 'ailerons', 'cd_airlines_700k', 'cpu_act', 'calHousing', 'yprop_4_1', 'house_16H', 'sulfur']


In [68]:
get_metrics("OnlineQRF", "cpu_act")

{'MER': [0.0206324014161885,
  0.022829935294835794,
  0.019411549261384446,
  0.016237333658893908,
  0.015871078012452692,
  0.01904529361494323,
  0.016115248443413503,
  0.018312782322060798,
  0.02136491270907093,
  0.018923208399462824],
 'RIS': [0.24837188883092925,
  0.26519375170333537,
  0.2576195356075713,
  0.3045384870558842,
  0.26176056746194704,
  0.28435373142979053,
  0.2514585483697924,
  0.23548018334979634,
  0.26148926698310165,
  0.24045238121663468],
 'QL': [0.025836067918841698,
  0.02761567574166768,
  0.02669054110880506,
  0.031542750172954055,
  0.02698625863074649,
  0.02927393825941012,
  0.025932626225630747,
  0.02448770453898033,
  0.02729948736541338,
  0.024933130597884597],
 'Utility': [0.7516281111690708,
  0.7348062482966646,
  0.7423804643924288,
  0.6954615129441157,
  0.738239432538053,
  0.7156462685702094,
  0.7485414516302076,
  0.7645198166502036,
  0.7385107330168983,
  0.7595476187833653],
 'time': [24.223845000000438,
  24.16984299999967

In [18]:
os.listdir("results/house_8L/CPExact")

['2022-06-06-18-57-23-613124',
 '2022-06-06-18-34-56-752334',
 '2022-06-06-21-32-34-406158',
 '2022-06-06-20-03-28-883516',
 '2022-06-06-20-25-02-193332',
 '2022-06-06-15-49-21-745165',
 '2022-06-06-19-19-50-933149',
 '2022-06-06-21-10-44-837727',
 '2022-06-06-19-41-20-960158',
 '2022-06-06-20-48-10-283244']

In [ ]:
def get_metrics(dataset_name):
    filepath = "results/" + dataset_name +"/metrics/"

In [41]:
def get_metrics(model_name, dataset_name):
    metric_files = []
    directories = os.listdir("results/"+dataset_name+"/"+model_name)
    for exp_id in directories:
        if exp_id != ".DS_Store":
            filepath = "results/"+dataset_name+"/"+model_name + "/" +exp_id +"/metrics.pckl"
            with open(filepath, "rb") as file:
                metric_file = pickle.load(file)
                metric_files.append(metric_file)
        
    final_metrics = dict()
    for key in metric_files[0].keys():
        final_metrics[key] = [metric[key] for metric in metric_files]
    return final_metrics
    

In [33]:
get_metrics("CPExact", "house_8L")

{'MER': [0.10248869771320722,
  0.09906509239345125,
  0.10718518193389809,
  0.0938418996620287,
  0.09726550498178467,
  0.1054733792740201,
  0.10077689505332924,
  0.10863363033841021,
  0.106746258174955,
  0.10600008778475178],
 'RIS': [2367883767050.1577,
  1792477851765.6868,
  2473170084638.309,
  2747999682446.366,
  3079462843371.34,
  2806850075632.1113,
  2405980812865.065,
  2875540435203.9053,
  4858772149128.247,
  2474293565964.9365],
 'QL': [1806886645302.6477,
  1962269752584.442,
  1873030181765.418,
  1560421440698.9248,
  2260763574875.903,
  2022367013525.506,
  1789897735379.291,
  2056568954300.5918,
  2622046510242.3467,
  1759059124720.4258],
 'Utility': [-2287583350526.961,
  -1792477851764.6868,
  -2238694740358.294,
  -2747999682445.366,
  -3079462843370.34,
  -2601753749875.893,
  -2380207313735.4385,
  -2551172617583.173,
  -4424966753569.257,
  -2276811612641.4917],
 'time': [89.86963700000024,
  96.15302999999994,
  82.87015500000052,
  85.035037999999

In [ ]:
def get_metrics_avg(model_name, dataset_name):
    metrics = get_metrics(model_name, dataset_name)
    for key in metrics.keys():
        metrics[key] = 

## Figuring out what's up with the crazy metrics

In [ ]:
def relative_interval_size(results):
    filtered_results = remove_inf_results(results)
    y_vals = [item[1] for item in filtered_results]
    rho = max(y_vals) - min(y_vals)
    # sum length of intervals
    s = sum(x[3][1] - x[3][0] for x in filtered_results)
    n = len(filtered_results)
    return s/(rho*n)

In [81]:
model_name = "OnlineQRF"
dataset_name = "house_8L"
for exp_id in os.listdir("results/"+dataset_name+"/"+model_name):
    filepath = "results/"+dataset_name+"/"+model_name + "/" +exp_id +"/results.pckl"
    with open(filepath, "rb") as file:
        results_file = pickle.load(file)
    break

In [83]:
results_file["OnlineQRF"]

[[{'x1': 1.0,
   'x2': 1.0,
   'x3': 1.0,
   'x4': 1.0,
   'x5': -1.0,
   'x6': 0.0,
   'x7': 1.0,
   'x8': 0.0,
   'x9': -1.0,
   'x10': 1.0},
  7.73906,
  0.0,
  [-inf, inf],
  0.1],
 [{'x1': -1.0,
   'x2': 0.0,
   'x3': -1.0,
   'x4': -1.0,
   'x5': 1.0,
   'x6': 1.0,
   'x7': 1.0,
   'x8': 0.0,
   'x9': -1.0,
   'x10': 0.0},
  3.95676,
  -0.17025932,
  [7.739059925079346, 7.739059925079346],
  0.1],
 [{'x1': 1.0,
   'x2': 0.0,
   'x3': 1.0,
   'x4': 1.0,
   'x5': -1.0,
   'x6': -1.0,
   'x7': 0.0,
   'x8': 1.0,
   'x9': 1.0,
   'x10': 1.0},
  4.71592,
  0.350172596,
  [3.9567599296569824, 7.739059925079346],
  0.1],
 [{'x1': 1.0,
   'x2': 0.0,
   'x3': 0.0,
   'x4': 1.0,
   'x5': 0.0,
   'x6': -1.0,
   'x7': 1.0,
   'x8': 1.0,
   'x9': 1.0,
   'x10': -1.0},
  5.02863,
  0.6752285012000001,
  [3.9567599296569824, 7.739059925079346],
  0.1],
 [{'x1': -1.0,
   'x2': 0.0,
   'x3': -1.0,
   'x4': -1.0,
   'x5': -1.0,
   'x6': -1.0,
   'x7': -1.0,
   'x8': 1.0,
   'x9': 1.0,
   'x10': -1